# Inference for flu data

## Flu case incidences in season 2010

In [ ]:
import numpy as np
import pandas as pd

import seirmo as se

flu_data = se.DatasetLibrary().french_flu()
flu_data = flu_data.loc[flu_data['year'] == 2010]

In [ ]:
# Plot incidences data
plot = se.IncidenceNumberPlot()
plot.add_data(flu_data, time_key='time_index', inc_key='inc')
plot.show()

## Model parameters inference

In [ ]:
import pints

class SEIRModel(pints.ForwardModel):
    def __init__(self):
        super(SEIRModel, self).__init__()

        self._model = se.SEIRModel()
    
    def n_outputs(self):
        # Returns number of model outputs.
        # Returns only incidence number, in line with the flu data
        return 1

    def n_parameters(self):
        # Returns number of parameters, i.e. 4 initial condition and 3 parameters.
        return 7
    
    def simulate(self, log_parameters, times):
        output = self._model.simulate(parameters=np.exp(log_parameters), times=times, return_incidence=True)
        n_incidence = output[:,4]

        return n_incidence

In [ ]:
# Guess on starting points of samplers
parameter_guess = np.array([1e5, 5.4, 6.8e4, 0, 0.5, 0.5, 0.1])

In [ ]:
# User of pints' SingleOutputProblem and GaussianLogLikelihood

In [ ]:
# Create log-likelihood
pints_model = SEIRModel()
problem = pints.SingleOutputProblem(pints_model, times, flu_data['inc'])
log_likelihood = pints.GaussianLogLikelihood(problem)

# Run sampling routines
optimiser = pints.OptimisationController(
    function=log_likelihood,
    x0=parameter_guess,
    method=pints.CMAES
)

In [ ]:
optimiser.set_max_iterations(1000)

log_estimates, _ = optimiser.run()
estimates = np.exp(log_estimates)

In [ ]:
print('Parameter names:')
print(['Initial S', 'Initial E', 'Initial I', 'Initial R', 'Infection Rate', 'Incubation Rate', 'Recovery Rate'])
print('Parameter estimates:')
print(estimates)

## Compare inferred model to data

In [ ]:
inferred_result = model.simulate(parameters=estimates, times=times, return_incidence=True)
inferred_data = pd.DataFrame({
            'Time': flu_data['time_index'],
            'Incidence Number': inferred_result[:, -1],
            'Susceptible': inferred_result[:, 0],
            'Exposed': inferred_result[:, 1],
            'Infectious': inferred_result[:, 2],
            'Recovered': inferred_result[:, 3],
        })

# Plot flu data vs inferred incidences data
plot = se.SubplotFigure()
plot.add_data(flu_data, time_key='time_index', inc_key='inc')
plot.show()
plot.add_simulation(inferred_data)
plot.show()